# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Passo 1: Abrir o navegador
navegador = webdriver.Chrome()

In [43]:
# Passo 2: Pesquisar no google a cotação do dólar
# Passo 3: Pegar a cotação do dólar

navegador.get('https://www.google.com/webhp?hl=en&sa=X&ved=0ahUKEwiBxorbmqfwAhU_GbkGHQMAC60QPAgI')
navegador.find_element_by_xpath(r'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do Dólar")
# .click()
# .send_keys()
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

5.438566


In [44]:
# Passo 4: Pesquisar no google a cotação do euro
# Passo 5: Pegar a cotação do euro

navegador.get('https://www.google.com/webhp?hl=en&sa=X&ved=0ahUKEwiBxorbmqfwAhU_GbkGHQMAC60QPAgI')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do Euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

6.537129139


In [45]:
# Passo 6: entrar no site melhorcambio
# Passo 7: Pegar a cotação do ouro

navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

309.14


In [46]:
# navegador.find_element_by_xpath('//*[@id="dock"]/ul/li[1]/a/img').click()
# import time
# time.sleep(10)

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [47]:
import pandas as pd

tabela_produtos = pd.read_excel("Produtos.xlsx")
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [48]:
# Passo 8: Atualizar as cotações
tabela_produtos.loc[tabela_produtos["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Passo 9: Atualizar o preço base reais -> cotação * preço base original
tabela_produtos["Preço Base Reais"] = tabela_produtos["Cotação"] * tabela_produtos["Preço Base Original"]

# Passo 10: Atualizar o preço final -> preço base reais * ajuste
tabela_produtos["Preço Final"] = tabela_produtos["Preço Base Reais"] * tabela_produtos["Ajuste"]

# Passo 11: Formatar os valores
tabela_produtos["Preço Final"] = tabela_produtos["Preço Final"].map("{:.2f}".format)
tabela_produtos["Cotação"] = tabela_produtos["Cotação"].map("{:.3f}".format)
tabela_produtos["Preço Base Reais"] = tabela_produtos["Preço Base Reais"].map("{:.2f}".format)

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.439,5438.51,1.40,7613.92
1,Carro Renault,4500.00,Euro,6.537,29417.08,2.00,58834.16
2,Notebook Dell,899.99,Dólar,5.439,4894.66,1.70,8320.91
3,IPhone,799.00,Dólar,5.439,4345.41,1.70,7387.20
4,Carro Fiat,3000.00,Euro,6.537,19611.39,1.90,37261.64
5,Celular Xiaomi,480.48,Dólar,5.439,2613.12,2.00,5226.24
6,Joia 20g,20.00,Ouro,309.140,6182.80,1.15,7110.22


### Agora vamos exportar a nova base de preços atualizada

In [49]:
tabela_produtos.to_excel("Produtos Atualizados.xlsx", index=False)